快速指导
======
启动和运行Transformers！无论你是开发者还是日常用户，这个快速指导将帮助你入门，并告诉你如何使用pipeline()用于推理，用AutoClass加载一个预训练的模型和预处理器，用PyTorch或TensorFlow快速训练一个模型。如果你是一个初学者，我们建议查看我们的教程或下一个课程，以便对这里介绍的概念进行更深入的了解。

在你开始之前，确保你已经安装了所有必要的库：

In [ ]:
!pip install transformers datasets

你还需要安装你喜欢的机器学习框架：

In [ ]:
pip install torch

## Pipeline

pipeline()是使用预训练模型进行推理的最简单和最快的方法。你可以将pipeline()开箱即用，用于不同模式的许多任务，其中一些任务如下表所示：

| **Task**                     | **Description**                                                                                              | **Modality**    | **Pipeline identifier**                       |
|------------------------------|--------------------------------------------------------------------------------------------------------------|-----------------|-----------------------------------------------|
| Text classification          | assign a label to a given sequence of text                                                                   | NLP             | pipeline(task=“sentiment-analysis”)           |
| Text generation              | generate text given a prompt                                                                                 | NLP             | pipeline(task=“text-generation”)              |
| Summarization                | generate a summary of a sequence of text or document                                                         | NLP             | pipeline(task=“summarization”)                |
| Image classification         | assign a label to an image                                                                                   | Computer vision | pipeline(task=“image-classification”)         |
| Image segmentation           | assign a label to each individual pixel of an image (supports semantic, panoptic, and instance segmentation) | Computer vision | pipeline(task=“image-segmentation”)           |
| Object detection             | predict the bounding boxes and classes of objects in an image                                                | Computer vision | pipeline(task=“object-detection”)             |
| Audio classification         | assign a label to some audio data                                                                            | Audio           | pipeline(task=“audio-classification”)         |
| Automatic speech recognition | transcribe speech into text                                                                                  | Audio           | pipeline(task=“automatic-speech-recognition”) |
| Visual question answering    | answer a question about the image, given an image and a question                                             | Multimodal      | pipeline(task=“vqa”)                          |
| Document question answering  | answer a question about a document, given an image and a question                                            | Multimodal      | pipeline(task="document-question-answering")  |
| Image captioning             | generate a caption for a given image                                                                         | Multimodal      | pipeline(task="image-to-text")                |

首先创建一个pipeline()的实例，并指定一个你想使用它的任务。在本指南中，你将以情感分析的pipeline()为例：

In [ ]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")

pipeline()下载并缓存了一个默认的预训练模型和标记器，用于情感分析。现在你可以在你的目标文本上使用该分类器：

In [ ]:
classifier("We are very happy to show you the 🤗 Transformers library.")

[{'label': 'POSITIVE', 'score': 0.9998}]

如果你有一个以上的输入，把你的输入作为一个列表传给pipeline()，以返回一个字典的列表：

In [ ]:
results = classifier(["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."])
    for result in results:
        print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: POSITIVE, with score: 0.9998

label: NEGATIVE, with score: 0.5309

pipeline()也可以为你喜欢的任何任务在整个数据集上迭代。在这个例子中，让我们选择自动语音识别作为我们的任务：

In [ ]:
import torch
from transformers import pipeline

speech_recognizer = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h")

加载一个你想迭代的音频数据集（详见《数据集快速入门》）。例如，加载MInDS-14数据集：

In [ ]:
from datasets import load_dataset, Audio

dataset = load_dataset("PolyAI/minds14", name="en-US", split="train")

你需要确保数据集的采样率与facebook/wav2vec2-base-960h训练的采样率一致：

In [ ]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=speech_recognizer.feature_extractor.sampling_rate))

调用 "音频 "列时，音频文件被自动加载并重新采样。从前4个样本中提取原始波形阵列，并将其作为一个列表传递给管道：

In [ ]:
result = speech_recognizer(dataset[:4]["audio"])
print([d["text"] for d in result])

['I WOULD LIKE TO SET UP A JOINT ACCOUNT WITH MY PARTNER HOW DO I PROCEED WITH DOING THAT', "FONDERING HOW I'D SET UP A JOIN TO HELL T WITH MY WIFE AND WHERE THE AP MIGHT BE", "I I'D LIKE TOY SET UP A JOINT ACCOUNT WITH MY PARTNER I'M NOT SEEING THE OPTION TO DO IT ON THE APSO I CALLED IN TO GET SOME HELP CAN I JUST DO IT OVER THE PHONE WITH YOU AND GIVE YOU THE INFORMATION OR SHOULD I DO IT IN THE AP AN I'M MISSING SOMETHING UQUETTE HAD PREFERRED TO JUST DO IT OVER THE PHONE OF POSSIBLE THINGS", 'HOW DO I FURN A JOINA COUT']

对于输入量较大的数据集（如语音或视觉），你要传递一个生成器而不是一个列表，以便在内存中加载所有的输入。更多信息请看 pipeline API reference。